# lady testing tea

In [1]:
import math 


In [2]:
no_cups = 8 
no_cups_milk_first = 4 
no_cups_tea_first = 4 


In [3]:
ways = math.comb(no_cups, no_cups_milk_first)
ways 


70

In [4]:
ways_ordered = 8 * 7 * 6 * 5 

ways_ordered

1680

In [5]:
no_shuffles = 4 * 3 * 2 * 1 
no_shuffles

24

In [6]:
no_combs = ways_ordered // no_shuffles
no_combs

70

-------------------------------------

In [7]:
import numpy as np 
from scipy.stats import fisher_exact 

In [9]:
cups_total = 8 
cups_milk_first = 4 
cups_tea_first = 4 

In [10]:
np.random.seed(42)
actual_cups = np.array([1]*cups_milk_first + [0]*cups_tea_first)
np.random.shuffle(actual_cups) 


In [11]:
lady_guesses = np.array([1]*cups_milk_first + [0]*cups_tea_first)

# Now we will count her correct guesses:
# Number of true positives (correct milk first identification) and false positives
true_positives = np.sum((actual_cups == 1) & (lady_guesses == 1))
false_positives = np.sum((actual_cups == 0) & (lady_guesses == 1))

# Create the contingency table
# Table of the format [[True Positives, False Positives], [False Negatives, True Negatives]]
contingency_table = np.array([[true_positives, false_positives], 
                              [cups_milk_first - true_positives, cups_tea_first - false_positives]])


In [12]:
oddsratio, p_value = fisher_exact(contingency_table, alternative='greater')

contingency_table, p_value

(array([[2, 2],
        [2, 2]]),
 0.7571428571428571)